# Protect against leakage

Now that we know how much that we're leaking privacy-wise, we can apply the `DiffPrivLib` library
to our model and see how the ROC curve flattens.
`DiffPrivLib` has an implementation of multiple models with an application of Differential Privacy:

- Classification models
  - Gaussian Naive Bayes
  - Logistic Regression
  - Tree-Based Models
- Regression models
  - Linear Regression
- Clustering models
  - K-Means
- Dimensionality reduction models
  - PCA
  
This notebook shows how to use the `RandomForestClassifier` from `DiffPrivLib` to
reduce the population metric.

## First exercices

1. What is surprising with regard to the population metric ROC when comparing the
two RandomForestClassifiers? Any idea why that is?
1. Compare the figures (feature values, population metric ROC, classifier ROC) 
for different values of `epsilon` to see how it behaves
1. Compare the figures (feature values, population metric ROC, classifier ROC) 
for the other dataset with `epsilon = 1`


## Further exercices

1. Find an optimal `n_estimators` and `max_depth` for maximizing
the AUROC of the classification, while minimizing the AUROC of
the population metric
1. Draw a series of (feature values, population metric ROC, classifier ROC)
for epsilon going from 10, 3, 1, 0.8, 0.5, 0.2

## NN exercices - no support

Use [Opacus](https://opacus.ai) to protect against leakage

In [ ]:
%run 2.1-population-metric.ipynb

In [ ]:
from diffprivlib.models import RandomForestClassifier as dp_RFC

def train_model_dp(X_train, y_train, seed=42):
    rfc_dp = dp_RFC(
        # Value from exercise: 90
        n_estimators=90,
        # Value from exercise: 9
        max_depth=9,
        random_state=42,
        # Value from exercise: 1
        epsilon=100,
        bounds=(np.min(X_train, axis=0), np.max(X_train, axis=0)),
        classes=np.unique(y_train),
    )
    return rfc_dp.fit(X_train, y_train)

target_model_dp = train_model_dp(X_train, y_train)
# The classifier's accuracy vs. random baseline. We are doing a bit better than the baseline.
print(f"Baseline: {max(y_test.mean(), 1 - y_test.mean()):0.2f}")
print(f"Our test-score: {target_model.score(X_test, y_test):0.2f}" )

In [ ]:
# Extract the features for the membership inference attack.
logits_train_dp = logits(target_model_dp, X_train, y_train)
logits_test_dp = logits(target_model_dp, X_test, y_test)

plot_rfc_auroc(y_test, target_model_dp.predict_proba(X=X_test)[:,1], "ROC of classifier using DP")
    
# Visualize the features. If it is possible to tell train data from test data, then
# our model is vulnerable to membership inference.
visualize_vals(logits_train_dp, logits_test_dp)

In [ ]:
membership_labels = np.concatenate([[1] * len(logits_train_dp), [0] * len(logits_test_dp)])
plot_rfc_auroc(membership_labels, np.concatenate([logits_train_dp, logits_test_dp]),
               "ROC of Population attack metric")